In [1]:
from multiprocessing import freeze_support
import pandas as pd
import string
import nltk
import re
import matplotlib.pyplot as plt
import gensim.corpora as corpora
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import numpy as np
from gensim.models import Phrases, CoherenceModel
from nltk.tokenize import word_tokenize
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from gensim.corpora import Dictionary
import pickle
from gensim.test.utils import datapath

from ipynb.fs.full.MLDRS_preprocessing import remove_punc, tokenization, remove_stopwords, lemmization, cleanResume, preprocessing, TokenizeLemm
from ipynb.fs.full.MLDRS_tools import createList, similarity, find_optimal_clusters, plot_tsne_pca


nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')
nltk.download('punkt')

wordnet_lemmatizer = WordNetLemmatizer()
wpt = nltk.WordPunctTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\

In [2]:
def Sort_Tuple(tup):   
    # key is set to sort using second element of 
    # sublist lambda has been used 
    tup.sort(key = lambda x: x[1], reverse = True) 
    return tup 

In [3]:
resumes = pd.read_csv(r"C://Users//zesha//Downloads//all_resumes.csv")
print("Number of resumes: " + str(resumes.shape[0]))

Number of resumes: 29783


In [4]:
with open('C://Users//zesha//Desktop//model', 'rb') as f:
     lda_model = pickle.load(f)
        
with open('C://Users//zesha//Desktop//jobs', 'rb') as f:
     jobs = pickle.load(f)

In [5]:
resumes = resumes.drop(columns=['Unnamed: 0'])
resumes.head()
resume = resumes.iloc[0]['resumes']

In [6]:
resume1 = remove_punc(resume)
resume1 = tokenization(resume1)
resume1 = remove_stopwords(resume1)
resume1 = lemmization(resume1)

In [7]:
dct = Dictionary([resume1])
resume_updated = dct.doc2bow(resume1)

In [8]:
resume_topics = lda_model[resume_updated]

In [9]:
resume_topics = (Sort_Tuple(resume_topics[0])) 
print(resume_topics)

[(11, 0.51847017), (7, 0.17254974), (6, 0.11415095), (9, 0.10110185), (5, 0.04490433), (1, 0.038097467)]


In [10]:
# jobs
city, citycount = np.unique(list(jobs.loc[:, "inferred_city"]), return_counts = True)
state, statecount  = np.unique(list(jobs.loc[:, "inferred_state"]), return_counts = True)
country, countrycount = np.unique(list(jobs.loc[:, "inferred_country"]), return_counts = True)

cities_dict = dict(zip(city, citycount))

In [11]:
city_entered = input("Enter your city: ")
for key,val in cities_dict.items():
    if key.lower() == city_entered.lower():
        new_corpus  = jobs[jobs["inferred_city"] == key]


if new_corpus.shape[0] < 5:
    print("Not enough data for this location.")

Enter your city: aberdeen


In [12]:
len(new_corpus)

30

In [13]:
for i in range (len(new_corpus)):
    new_corpus.iloc[i]['Topics'] = Sort_Tuple(new_corpus.iloc[i]['Topics'])

C:\Users\zesha\Anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [14]:
similarity_list = []

for i in range (len(new_corpus)):
    resume_temp = []
    job_temp = []
    for key, value in resume_topics:
        resume_temp.append(key)
    for key, value in new_corpus.iloc[i]['Topics']:
        job_temp.append(key)
        
    similarity_val = similarity(resume_temp, job_temp)
    similarity_list.append((similarity_val))
    
new_corpus['Similarity'] = similarity_list

C:\Users\zesha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
topNindexes = sorted(range(len(similarity_list)), key=lambda i: similarity_list[i], reverse=True)[:5]
topNindexes

[6, 24, 14, 0, 11]

In [16]:
top_5_idx = np.argsort(similarity_list)[-5:]
top_5_values = [similarity_list[i] for i in top_5_idx]

In [20]:
top_5_idx

array([12,  0, 14, 24,  6], dtype=int64)

In [17]:
top_5_values

[37.0, 39.0, 42.0, 44.0, 69.0]

In [18]:
new_corpus_vals = new_corpus.index[topNindexes]
new_corpus_vals

Int64Index([2039, 14518, 7987, 37, 7616], dtype='int64')

In [19]:
topN = new_corpus.loc[new_corpus_vals]
topN

,Unnamed: 0,job_title,category,company_name,inferred_city,inferred_state,inferred_country,job_description,job_type,Topics,Similarity
2039,2039,Software Developer,architecture and engineering,"Data Systems Analysts, Inc",Aberdeen,Maryland,Usa,"Data Systems Analysts, Inc. is searching for a...",Full Time,"[(11, 0.34015608), (4, 0.1403401), (3, 0.11725...",69.0
14518,4525,Safety Data Scientist,Manufacturing-or-mechanical,AASKI Technology,Aberdeen,Maryland,United states,Overview\n\nAASKI Technology is a dynamic gove...,Undefined,"[(10, 0.28475037), (4, 0.24056295), (5, 0.1893...",44.0
7987,7987,Junior Software Developer,computer jobs,"Applied Research Associates, Inc",Aberdeen,Maryland,United states,Looking for a great company where you can be a...,Undefined,"[(4, 0.30075344), (5, 0.24569768), (0, 0.18960...",42.0
37,37,Remote Software Developer,NaN,Capital Markets Placement,Aberdeen,Maryland,Usa,Precision is king when it comes to the work yo...,Full Time,"[(4, 0.42233145), (11, 0.28008574), (1, 0.0879...",39.0
7616,7616,Software Developer Int.,Computer-or-internet,Jacobs,Aberdeen,Maryland,United states,Overview:\n\nJacobs is one of the world's larg...,Undefined,"[(4, 0.4667218), (5, 0.3752453), (11, 0.097968...",37.0
